In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

# Load the data
path = '/content/gdrive/MyDrive/att_faces'
folder_names = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10',
                's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20',
                's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30',
                's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39', 's40']
                
train_file_names = ['1.pgm', '2.pgm', '3.pgm', '4.pgm', '5.pgm', 
                    '6.pgm', '7.pgm', '8.pgm', '9.pgm']

test_file_names = ['10.pgm']


import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.image as img

Mounted at /content/gdrive


## **4.1 Creating Training and Data Sets**

In [ ]:
#Open Each Folder
samples = 9
testSamples = 1
Xtest = np.zeros((10304,40))
Xtrain = np.zeros((10304,360))
Ytrain = np.zeros(40*samples)
Ytest = np.zeros(40*testSamples)

for folder in folder_names:
  i = folder_names.index(folder)
  for File in train_file_names:
      j = train_file_names.index(File)
      im_path = path + '/' + folder + '/' + File
      current = img.imread(im_path)
      Xtrain[:, i*samples + j] = np.ravel(current)
      Ytrain[i*samples + j] = int(i + 1)
  for File in test_file_names:
      j = test_file_names.index(File)
      im_path = path + '/' + folder + '/' + File
      current = img.imread(im_path)
      Xtest[:, i*testSamples + j] = np.ravel(current)
      Ytest[i*testSamples + j] = int(i + 1)

In [ ]:
print(Xtest.shape)
print(Xtrain.shape)
print(Ytest.shape)
print(Ytrain.shape)

(10304, 40)
(10304, 360)
(40,)
(360,)


## **5.1 PCA Transform of Training Points**

In [ ]:
def train(Xtrain, K):
  N, n_faces = Xtrain.shape
  mu = Xtrain.mean(axis=1) 
  X = np.zeros((N, n_faces))
  for i in range(n_faces):
    X[:, i] = Xtrain[:, i] - mu

  sigma = np.matmul(X, X.transpose()) 
  sigma /= n_faces
  _, sigma_eigs = sc.sparse.linalg.eigs(sigma, k= K)
  P_H = np.conj(np.transpose(sigma_eigs))
  X_pca = np.matmul(P_H, X)
  return mu, sigma_eigs, X_pca

##**5.2 PCA Transform of Test**

In [ ]:
def test(mean, P, Xtest):
  N, n_faces = Xtest.shape
  X = np.zeros((N, n_faces))

  for i in range(n_faces):
    X[:, i] = Xtest[:, i] - mean
    
  P_H = np.conj(np.transpose(P))
  X_pca = np.matmul(P_H, X)

  return X_pca

## **6.1 Nearest Neighbor Class**

In [ ]:
def nearest_neighbor(Xtrain, Xtest):
    distances = sc.spatial.distance.cdist(Xtrain.transpose(), Xtest.transpose())
    predicted = np.argmin(distances, axis=0)
    return predicted

for K in [1, 5, 10, 20]:
  mean, P, train_PCA = train(Xtrain, K)
  test_PCA = test(mean, P, Xtest)
  predicted = nearest_neighbor(Xtrain, Xtest)
  for pred in predicted:
    predicted_folder = int(pred / 40) + 1
    predicted_image = int(pred % 9) + 1

In [ ]:
n_examples = 3
n_samples = 9

im_dataset_path = "/content/gdrive/MyDrive/att_faces"

training_set, training_labels, test_set, test_labels = training_and_test_sets(n_samples, im_dataset_path)


for k in [1, 5, 10, 20]:

  for jj in range(n_examples):
              # Displaying images
              im_number = np.random.randint(1, 40)  # random image from the test set
              im_path = im_dataset_path + '/s' + str(im_number) + '/' + str(10) + '.pgm'
              og_img = mpimg.imread(im_path)
              predicted_person = int(training_labels[neighbor[im_number-1]])
              s_number = neighbor[im_number - 1] - (predicted_person-1)*n_samples
              im_path = im_dataset_path + '/s' + str(predicted_person) + '/' + str(s_number+1) + '.pgm'
              nearest_img = mpimg.imread(im_path)
              fig = plt.figure()
              plt.suptitle('Original and Nearest Images: k = ' + str(k), fontsize=10) #str( is the # of k = 1,2,5,10)
              ax1 = fig.add_subplot(1,2,1)
              ax1.imshow(og_img, cmap='gray')
              ax1.axes.xaxis.set_visible(False)
              ax1.axes.yaxis.set_visible(False)
              plt.title('Original', fontsize=9)
              ax2 = fig.add_subplot(1,2,2)
              ax2.imshow(nearest_img, cmap='gray')
              ax2.axes.xaxis.set_visible(False)
              ax2.axes.yaxis.set_visible(False)
              plt.title('Nearest', fontsize=9)
              # Saving plot
              fig_path_name = 'k_' + str(k) + '_ex_' + str(jj) + '.png'
              plt.savefig(fig_path_name, dpi=300)
              plt.show()